In [ ]:
import os
import numpy as np
# import cv2
from random import shuffle
from tensorflow.keras import layers
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from collections import Counter
from tensorflow import keras

#import util_layers
#import swin_layers
#import transformer_layers
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Layer, Dropout, Flatten
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
from vit_keras import vit
warnings.filterwarnings("ignore")



# Preparing dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
batchsize=16
image_size=224

visible = tf.keras.layers.Input(shape=(image_size, image_size, 3))
train_dir = 'C:/my data/datasetV3/train'
validation_dir = 'C:/my data/datasetV3/validation'
test_dir = 'C:/my data/datasetV3/test'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   #shear_range = 0.2,
                                   #zoom_range = 0.2,
                                   #validation_split=0.2,
                                   #horizontal_flip = True
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (image_size, image_size),
                                                 batch_size = batchsize,
                                                 #subset="training",
                                                 class_mode = 'categorical')

validation_set = train_datagen.flow_from_directory(validation_dir,
                                                 target_size = (image_size, image_size),
                                                 batch_size = batchsize,
                                                #subset="validation",
                                                shuffle = False,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                       batch_size = batchsize,
                                       shuffle = False,

                                       target_size = (image_size, image_size))


# main fuctons

In [ ]:

def display_training_curves(training_accuracy, validation_accuracy, title, subplot):
    if subplot % 10 == 1:
        plt.subplots(figsize=(10, 10))
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.plot(training_accuracy)
    ax.plot(validation_accuracy)
    ax.set_title('Model ' + title)
    ax.set_ylabel(title)
    ax.set_xlabel('Epoch')
    ax.legend(['Train_Accuracy', 'Val_Accuracy'])


def display_training_curves2(training_loss, validation_loss, title, subplot):
    if subplot % 10 == 1:
        plt.subplots(figsize=(10, 10))
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.plot(training_loss)
    ax.plot(validation_loss)
    ax.set_title('Model ' + title)
    ax.set_ylabel(title)
    ax.set_xlabel('Epoch')
    ax.legend(['Train_Loss', 'Val_Loss'])


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # map the activations of the last conv layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap, top_pred_index.numpy()


def superimposed_img(image, heatmap):
    heatmap = np.uint8(255 * heatmap)
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((image_size, image_size))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + image
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    return superimposed_img


# label smoothing
def categorical_smooth_loss(y_true, y_pred, label_smoothing=0.1):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=label_smoothing)
    return loss




# build the hybrid model in parallel mode

In [ ]:
path='models_ouput/hybrid)ensemb(InceptionResNetV2_Xception)(parallel1)With_ViT'
import tensorflow_addons as tfa
image_size=224
#pretrained_E = tf.keras.applications.EfficientNetV2B0(input_shape=(image_size, image_size, 3), weights='imagenet', include_top=False)
#pretrained_VGG = tf.keras.applications.VGG16(input_shape=(image_size, image_size, 3), weights='imagenet',
#                                            include_top=False)
# pretrained_googleNet = tf.keras.applications.InceptionV3(input_shape=(image_size, image_size, 3), weights='imagenet',
#                                                          include_top=False)
pretrained_IRV2 = tf.keras.applications.InceptionResNetV2(input_shape=(image_size, image_size, 3), weights='imagenet',
                                                         include_top=False)
pretrained_Xception = tf.keras.applications.Xception(input_shape=(image_size, image_size, 3), weights='imagenet',
                                                     include_top=False)


for layer in pretrained_IRV2.layers:
    layer.trainable = False

for layer in pretrained_Xception.layers:
    layer.trainable = False



x4 = pretrained_IRV2(visible)
x4 = tf.keras.layers.ZeroPadding2D(padding=((0, 2), (0, 2)))(x4)
x4 = tf.keras.layers.AveragePooling2D()(x4)
x4 = tf.keras.layers.Flatten()(x4)
x4 = tf.keras.layers.Dense(32, activation="relu")(x4)
x4 = tf.keras.layers.Dropout(0.5)(x4)

x5 = pretrained_Xception(visible)
x5 = tf.keras.layers.AveragePooling2D()(x5)
x5 = tf.keras.layers.Flatten()(x5)
x5 = tf.keras.layers.Dense(32, activation="relu")(x5)
x5 = tf.keras.layers.Dropout(0.5)(x5)





vit_model = vit.vit_b32(
        image_size = image_size,
        pretrained = True,
        include_top = False,
        pretrained_top = False)



vit_x=vit_model(visible)
vit_x=tf.keras.layers.Flatten()(vit_x)
vit_x=tf.keras.layers.BatchNormalization()(vit_x)
vit_x=tf.keras.layers.Dense(32, activation = tfa.activations.gelu)(vit_x)

merge = tf.keras.layers.concatenate([x4, x5, vit_x])
x = tf.keras.layers.Dense(32, activation='relu')(merge)
x = tf.keras.layers.Dropout(0.5)(x)
OUT = tf.keras.layers.Dense(33, activation='softmax')(x)

model = Model(inputs=visible, outputs=OUT)


# build the hybrid model in serial mode

In [ ]:
path='models_ouput/hybrid)ensemb(InceptionResNetV2_Xception)(serial)With_ViT'
import tensorflow_addons as tfa


pretrained_IRV2 = tf.keras.applications.InceptionResNetV2(input_shape=(image_size, image_size, 3), weights='imagenet',
                                                         include_top=False)
pretrained_Xception = tf.keras.applications.Xception(input_shape=(image_size, image_size, 3), weights='imagenet',


for layer in pretrained_IRV2.layers:
    layer.trainable = False

for layer in pretrained_Xception.layers:
    layer.trainable = False


x4 = pretrained_IRV2(visible)
x4 = tf.keras.layers.ZeroPadding2D(padding=((0, 2), (0, 2)))(x4)
x4 = tf.keras.layers.AveragePooling2D()(x4)
x4 = tf.keras.layers.Flatten()(x4)
x4 = tf.keras.layers.Dense(32, activation="relu")(x4)
x4 = tf.keras.layers.Dropout(0.5)(x4)

x5 = pretrained_Xception(visible)
x5 = tf.keras.layers.AveragePooling2D()(x5)
x5 = tf.keras.layers.Flatten()(x5)
x5 = tf.keras.layers.Dense(32, activation="relu")(x5)
x5 = tf.keras.layers.Dropout(0.5)(x5)


vit_model = vit.vit_b32(
        image_size = image_size,
        pretrained = True,
        include_top = False,
        pretrained_top = False)


vit_x=vit_model(visible)
vit_x=tf.keras.layers.Flatten()(vit_x)
vit_x=tf.keras.layers.BatchNormalization()(vit_x)
vit_x=tf.keras.layers.Dense(32, activation = tfa.activations.gelu)(vit_x)

cnn_outputs = tf.keras.layers.concatenate([x4, x5])


merge = tf.keras.layers.concatenate([cnn_outputs, vit_x])
x = tf.keras.layers.Dense(32, activation='relu')(merge)
x = tf.keras.layers.Dropout(0.5)(x)
OUT = tf.keras.layers.Dense(33, activation='softmax')(x)
#with tf.device('/GPU:0'):
model = Model(inputs=visible, outputs=OUT)



In [ ]:
print(model.summary())

# train the model

In [ ]:
import numpy as np


IMAGE_SIZE = 224
BATCH_SIZE = 4
EPOCHS = 80
learning_rate = 1e-4
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)
print(model.summary)
model.compile(optimizer = optimizer,
              loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2),
              metrics = ['accuracy'])

STEP_SIZE_TRAIN = training_set.n // training_set.batch_size
STEP_SIZE_VALID = validation_set.n // validation_set.batch_size

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 4,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-6,
                                                 mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 10,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = path+'.h5',
                                                  monitor = 'val_accuracy',
                                                  verbose = 1,
                                                  save_best_only = True,

                                                  mode = 'max')


from keras.callbacks import ModelCheckpoint, LearningRateScheduler,CSVLogger
from keras.callbacks import ReduceLROnPlateau


csv_logger = CSVLogger(path+'.csv', append=True, separator=',')
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
#callbacks = [checkpoint, lr_reducer]

callbacks = [earlystopping, reduce_lr, checkpointer]

model.fit(x = training_set,
          validation_data = validation_set,
          epochs = EPOCHS,
          callbacks = [callbacks,csv_logger])



In [ ]:
import tensorflow as tf

from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
learning_rate = 1e-4
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30

optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)
model.load_weights('models_ouput/hybrid)ensemb(InceptionResNetV2_Xception)(parallel)With_ViT.h5')
#model.load_weights('models_ouput/hybrid)ensemb(InceptionResNetV2_Xception)(serial)With_ViT.h5')



In [ ]:
#this resylt of
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import numpy as np


# Make predictions on the test data
y_pred_prob = model.predict(test_set)

y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Get true labels from the test data generator
y_true = test_set.classes

# Calculate Accuracy (ACC)
accuracy = accuracy_score(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='macro')  # Specify 'macro' for multiclass
# Calculate Recall (SEN)
recall = recall_score(y_true, y_pred, average='macro')  # Specify 'macro' for multiclass

# Calculate F1-Score
f1 = f1_score(y_true, y_pred, average='macro')  # Specify 'macro' for multiclass

roc_auc = roc_auc_score(y_true, y_pred_prob, multi_class='ovr', average='macro')  # Specify 'ovr' and 'macro' for multiclass

# Print the results
print(" macro----------------------------")
print(f"Accuracy (ACC): {accuracy}")
print(f"Precision (PRE): {precision}")
print(f"Recall (SEN): {recall}")
print(f"F1-Score: {f1}")
print(f"AUC (Area Under ROC Curve): {roc_auc}")

cm = confusion_matrix(y_true, y_pred)

# Calculate the classification report
#cr = classification_report(y_true, y_pred)

# Calculate the F1-score
f1 = f1_score(y_true, y_pred, average='weighted')

# Calculate the accuracy
acc = accuracy_score(y_true, y_pred)

# Calculate the precision
pre = precision_score(y_true, y_pred, average='weighted')

recall = recall_score(y_true, y_pred, average='weighted')  # Specify 'macro' for multiclass

# Calculate the AUC
auc = roc_auc_score(y_true, y_pred_prob, multi_class='ovr', average='weighted')

# Calculate the sensitivity and specificity
#fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=None)
#sensitivity = tpr / (tpr + fpr)
#specificity = 1 - sensitivity
# Print the results
print("weighted-------------------------------")
print(f"Accuracy (ACC): {acc}")
print(f"Precision (PRE): {pre}")
print(f"Recall (SEN): {recall}")
print(f"F1-Score: {f1}")
print(f"AUC (Area Under ROC Curve): {auc}")
#print(f"Specificity (SPE): {specificity}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#hybrid)ensemb(InceptionResNetV2_Xception)(parallel)With_ViT)
history = pd.read_csv('models_ouput/hybrid)ensemb(InceptionResNetV2_Xception)(parallel)With_ViT.csv')
acc = history['accuracy'][:35]
val_acc = history['val_accuracy'][:35]
loss = history['loss'][:35]
val_loss = history['val_loss'][:35]

#plot results
#accuracy
plt.figure(figsize=(8, 8))
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
#plt.title(f'MobileNetV2 \nTraining and Validation Accuracy. \nTrain Accuracy: {str(acc[-1])}\nValidation Accuracy: {str(val_acc[-1])}')

#loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.title(f'Training and Validation Loss. \nTrain Loss: {str(loss[-1])}\nValidation Loss: {str(val_loss[-1])}')
plt.xlabel('epoch')
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming you already have the confusion_matrix_array and class_names
# Define your class names
class_names = [
    'Eight', 'Eight Hundred', 'Eighty', 'Fifty', 'Five', 'Five Hundred',
    'Forty', 'Four', 'Four Hundred', 'Hundred', 'Million', 'Nine',
    'Nine Hundred', 'Ninety', 'One', 'Only', 'Reyal', 'Seven', 'Seven Hundred',
    'Seventy', 'Six', 'Six Hundred', 'Sixty', 'Ten', 'Thirty', 'Thousand',
    'Three', 'Three Hundred', 'Twenty', 'Two', 'Two Hundred', 'Two Million', 'Two Thousand'
]

# Replace this line with your predicted class labels
predicted_classes = y_pred

true_classes = test_set.classes
# Create a confusion matrix
confusion_matrix_array = confusion_matrix(true_classes, predicted_classes)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_array, display_labels=class_names)

# Set x-axis labels rotation
fig, ax = plt.subplots(figsize=(14, 12))
disp.plot(cmap='Blues', values_format='d', ax=ax)
ax.set_xticklabels(class_names, rotation=90)  # Adjust rotation angle as needed

plt.title('Confusion Matrix')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
classes=['Eight', 'Eight Hundred', 'Eighty', 'Fifty', 'Five', 'Five Hundred', 'Forty', 'Four', 'Four Hundred', 'Hundred', 'Million', 'Nine', 'Nine Hundred', 'Ninety', 'One', 'Only', 'Reyal', 'Seven', 'Seven Hundred', 'Seventy', 'Six', 'Six Hundred', 'Sixty', 'Ten', 'Thirty', 'Thousand', 'Three', 'Three Hundred', 'Twenty', 'Two', 'Two Hundred', 'Two Million', 'Two Thousand']
true_labels = test_set.classes

# Initialize arrays to store true labels and predicted probabilities
true_labels_array = []
y_pred_prob_array = []

# Calculate ROC curve and AUC for each class
num_classes = len(test_set.class_indices)
fpr = {}
tpr = {}
roc_auc = {}

for i in range(num_classes):
    true_labels_i = (true_labels == i).astype(int)
    y_pred_i = y_pred_prob[:, i]

    fpr[i], tpr[i], _ = roc_curve(true_labels_i, y_pred_i)
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Store true labels and predicted probabilities for micro-average
    true_labels_array.extend(true_labels_i)
    y_pred_prob_array.extend(y_pred_i)

# Calculate micro-average ROC-AUC
micro_roc_auc = auc(fpr[0], tpr[0])

# Calculate PR curve and AP for each class
precision = {}
recall = {}
average_precision = {}

for i in range(num_classes):
    true_labels_i = (true_labels == i).astype(int)
    y_pred_i = y_pred_prob[:, i]

    precision[i], recall[i], _ = precision_recall_curve(true_labels_i, y_pred_i)
    average_precision[i] = average_precision_score(true_labels_i, y_pred_i)

# Calculate micro-average precision and recall directly
micro_precision, micro_recall, _ = precision_recall_curve(true_labels_array, y_pred_prob_array)
micro_average_precision = average_precision_score(true_labels_array, y_pred_prob_array)

# Calculate macro-average ROC-AUC and PR
macro_roc_auc = np.mean(list(roc_auc.values()))
macro_average_precision = np.mean(list(average_precision.values()))

# Plot ROC curves
plt.figure(figsize=(10, 6))
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'{classes[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower center',ncol=5)
plt.show()

# Plot PR curves
plt.figure(figsize=(10, 6))
for i in range(num_classes):
    plt.plot(recall[i], precision[i], lw=2, label=f'{classes[i]} (AP = {average_precision[i]:.2f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall (PR) Curve')
plt.legend(loc='lower center',ncol=5)
plt.show()

# Print micro and macro averages
print("Micro-average ROC-AUC:", micro_roc_auc)
print("Micro-average Precision:", micro_average_precision)
print("Micro-average Recall:", np.max(micro_recall))  # Use np.max to get the maximum recall for micro-average
print("Macro-average ROC-AUC:", macro_roc_auc)
print("Macro-average Precision:", macro_average_precision)
